В объектно-ориентированном программировании чисто технически объектом можно сделать что угодно. Иногда это окажется удобно, иногда жутко неудобно. Человечество подмечает закономерности и "удобные" называет паттерны (шаблоны) проектирования, а "неудобные" — антипаттерны. Также можно заметить более общие закономерности и сформулировать какие-нибудь _принципы проектирования_.

Есть популярная группа из пяти принципов проектирования объектно-ориентированных программ, которая называется [SOLID](https://ru.wikipedia.org/wiki/SOLID_(%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BD%D0%BE-%D0%BE%D1%80%D0%B8%D0%B5%D0%BD%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D0%BE%D0%B5_%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5)) — по первым буквам названий этих принципов. Сейчас я их перечислю, а в ближайших файлах разберём каждый подробно: сначала формулируется принцип, потом идёт пояснение, потом идёт пример антипаттерна (как делать **не** надо), а потом пример того, как можно принцип всё-таки соблюсти. Исходные формулировки могут звучать непонятно:

1. **S**ingle responsibility principle (SRP) — принцип единой ответственности. У каждого класса должна быть ровно одна ответственность и не больше.
2. **O**pen-closed principle (OCP). Каждая сущность должна быть открыта для расширения, но закрыта для модификации.
3. **L**iskov substitution principle (LSP). Объекты должно быть можно заменить на любой экземпляр его подтипа без нарушения корректности.
4. **I**nterface segregation principle (ISP). Много интерфейсов под конкретных клиентов лучше, чем один общий интерфейс.
5. **D**ependency inversion principle (DIP). Не абстракции зависят от реализаций, а реализации от абстракций.

Начнём с SRP. Типичный антипаттерн для него — [God Object](https://ru.wikipedia.org/wiki/%D0%91%D0%BE%D0%B6%D0%B5%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82). Это когда у нас есть один объект, который делает сразу всё. Например, пусть мы хотим сделать какой-нибудь отчёт:

In [1]:
import sys

class Report:
    def main(self):
        _, self.data, output = sys.argv
        if output == 'xml':
            self.make_xml_report()
        if output == 'markdown':
            self.make_markdown_report()
            
    def make_report(self):
        if self.data.startswith('!'):
            return {
                'Важно': [self.data]
            }
        return {
            'Секция 1': ['Привет', self.data],
            'Секция 2': ['Пока', self.data],
        }

    def make_xml_report(self):
        print('<xml>')
        for section, values in self.make_report().items():
            print('<section name="{}">'.format(section))
            for value in values:
                print('<value>{}</value>'.format(value))
            print('</section>')
        print('</xml>')
        
    def make_markdown_report(self):
        for section, values in self.make_report().items():
            print('# ' + section)
            for value in values:
                print('* ' + value)

In [2]:
sys.argv = [None, 'me', 'xml']
Report().main()

<xml>
<section name="Секция 1">
<value>Привет</value>
<value>me</value>
</section>
<section name="Секция 2">
<value>Пока</value>
<value>me</value>
</section>
</xml>


In [3]:
sys.argv = [None, 'me', 'markdown']
Report().main()

# Секция 1
* Привет
* me
# Секция 2
* Пока
* me


In [4]:
sys.argv = [None, '!me', 'markdown']
Report().main()

# Важно
* !me


В чём проблема этого объекта? Вообще объекты разрешают и провоцируют активно связывать вместе разные методы. Например, через поля. Или через вызовы друг друга. Как следствие, тут нельзя протестировать отдельно вывод отчёта, не протестировав при этом `make_report`. А его нельзя протестировать, не задав `data` или не вызвав `main()`. Это всё следствия.

Нарушается ли тут принцип SRP? Да, нарушается: этот объект и аргументы командной строки обрабатывает, и отчёт составляет, и форматирует его в нужный формат, и на экран выводит. То есть по-хорошему тут должно быть четыре объекта минимум. Мы не стремимся сделать объектами вообще всё, поэтому я бы парсинг командной строки и вывод на экран сделал просто отдельными функциями. А вот оставшиеся две ответственности (создание отчёта и вывод на экран) можно сделать объектами.

Тут естественным образом вылезает две абстракции: "создатель отчётов" (кто-то, кто возвращает отчёт в виде словаря, может быть хоть функцией) и "форматирователь отчётов" (кто-то, кто по отчёту генерирует результат, тоже может быть функцией, в целом).

In [5]:
from abc import ABCMeta, abstractmethod

class AbstractReportCreator(metaclass=ABCMeta):
    @abstractmethod
    def make_report(self):
        pass
    
class AbstractReportFormatter(metaclass=ABCMeta):
    @abstractmethod
    def format_report(self, report):
        pass
    
class BasicReportCreator(AbstractReportCreator):
    def __init__(self, data):
        self.data = data
    
    def make_report(self):
        return {
            'Секция 1': ['Привет', self.data],
            'Секция 2': ['Пока', self.data],
        }

class ImportantReportCreator(AbstractReportCreator):
    def __init__(self, data):
        self.data = data
    
    def make_report(self):
        return {
            'Важно': [self.data]
        }
    
class XmlReportFormatter(AbstractReportFormatter):
    def format_report(self, report):
        yield '<xml>'
        for section, values in report.items():
            yield '<section name="{}">'.format(section)
            for value in values:
                yield '<value>{}</value>'.format(value)
            yield '</section>'
        yield '</xml>'
        
class MarkdownReportFormatter(AbstractReportFormatter):
    def format_report(self, report):
        for section, values in report.items():
            yield '# ' + section
            for value in values:
                yield '* ' + value

def print_report(creator, formatter):
    for line in formatter.format_report(creator.make_report()):
        print(line)

Длинно? А то. Зато теперь мы можем:

1. Тестировать все кусочки по отдельности.
2. Переиспользовать создание и форматирование отчётов в других местах кода.
3. Безболезненно добавлять новые способы создания и форматирования отчётов. Хоть плагины поддержать.

Разумеется, все эти пункты в маленькой программе с чёткими требованиями неактуальны и ООП тут скорее раздует код. Зато если мы работаем над большим проектом, который будет развиваться в неизвестном направлении, оказывается удобным написать чуть больше кода на каждую реализации, зато получить прекрасный удобный способ композиции:

In [6]:
print_report(BasicReportCreator('me'), XmlReportFormatter())

<xml>
<section name="Секция 1">
<value>Привет</value>
<value>me</value>
</section>
<section name="Секция 2">
<value>Пока</value>
<value>me</value>
</section>
</xml>


In [7]:
print_report(ImportantReportCreator('!me'), MarkdownReportFormatter())

# Важно
* !me


Код выше намного чётче отражает намерения программиста и намекает, что у нас программа состоит из маленьких независимых кусочков. Гораздо проще понять, какой кусочек нужно поменять, если мы хотим добавить новый способ форматирования отчётов.